In [1]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys
import pandas as pd
from urllib.parse import quote_plus

Using TensorFlow backend.


In [2]:
# 파일을 읽어온다. 
fp = pd.read_csv('all_event.csv')

import csv
with open('all_event.csv' , 'rt') as fin:
    cin = csv.reader(fin) 
    all_event = [row for row in cin]
    
print(fp.shape)
# 사건갯수, 시간

(50000, 24)


In [3]:
results_ = []
for i , event in enumerate(all_event):
    # print(i , event)
    event = event[1:]
    
    if '' in event:
        event = event[:event.index('')]
        
    evt_txt = ' '.join(event) 
    results_.append(evt_txt)

In [4]:
results = results_[1:]
results

['No3368 En1339 Po1313 Te1354 En2411 No2563 Po3339 Bu2471 Ec3212 Se2421',
 'En1314 En1442 Se3242 Tr3139 Tr2346 Se3233 En1434 No1366 No2127 No1433 Tr1241 Se3252 Se3514',
 'No2112 Tr3443 Se1248 No2271 Se3329 Tr3257 Te2114 Ec3234 Tr1476 Te3367 Po2178 Se1335 Se3141 Tr1321 Po2559 Po2533 Te1441',
 'Bu1271 En3567 Te3328 Bu2155 Po2468 Se1428 Bu3143 Ec1262 Bu1435 Bu3511 No3478',
 'Se1126 Tr3317 Po2149 Te3473 En2146 En2335 No2457 Po3128 Ec2221 Tr1116',
 'Se1534 Po1237 Tr3568 No2458 Se2471 Se2525 Bu1554 Ec2325 En3462 Ec2258 Bu3242 Bu3359 Tr1411 En2526 Te3274 Te3357 No2353 No1143 Po1513 Te1377 Ec1439 No1255 Ec3422',
 'En2241 Te3472 No2358 Ec2518 En1317',
 'Se1311 En3523 Se2539 No3454 Tr3539 Se2214 Bu1454 No1325 Po3137 Tr2271 Bu3477',
 'No3459 Te3265 Bu1121 Bu1466 Tr3129 Te2252 Tr3531',
 'En3126 Bu1348 Te3453 Se1544 Se2469 Bu2551 Tr2439 Te3252',
 'Te1544 Se1413 No3162 Po3451 En1458 Po3447 Tr2547 Tr2457 Tr2575 Po1566 Se1236 Po1159 Te2132 Te2128',
 'Te3537 Te1564 Se1523 No3239 Ec2161 Te1177 Po2313 No

In [5]:
all_text = " ".join(results) 
text = all_text
print('훈련 데이터: ', len(text))

# 사건사전 생성
text_split = text.split(" ")
chars = sorted(list(set(text_split)))
print('사건의 종류:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))  
indices_char = dict((i, c) for i, c in enumerate(chars))  

# 전체 훈련 데이터를 쪼개어서 문장구조처럼 만들기
maxlen = 20  # 예측할 사건 시간 범위
step = 10
sentences = []
next_chars = []

# sentences에 maxlen개씩 엮어넣고
# next_chars에 그 다음 사건 넣기
for i in range(0, len(text_split) - maxlen, step):
    sentences.append(" ".join(text_split[i: i + maxlen]))
    next_chars.append(text_split[i + maxlen])
print("sentences:", len(sentences))


훈련 데이터:  4906670
사건의 종류: 7560
sentences: 70094


In [6]:
print('학습할 사건들의 수:', len(sentences))
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(next_chars), len(chars)), dtype=np.bool)
print("X:", X.shape)
print("y:", y.shape)

for i, sentence in enumerate(sentences):  # 학습할 사건들
    # one-hot encoding
    for t, char in enumerate(sentence.split(" ")):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

학습할 사건들의 수: 70094
X: (70094, 20, 7560)
y: (70094, 7560)


In [7]:
# 모델 구축하기(LSTM)
print('LSTM 모델을 만들어 본다....')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))  # 일련의 사건 목록 수, 사건 종류 수
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.015)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

for iteration in range(1, 10):
    print()
    print('반복번호 =', iteration)
    model.fit(X, y, batch_size=128, epochs=1)

# 출력결과보면 iteration마다 loss 변화가 없어서
# 반복 2번 정도면 충분할 듯

LSTM 모델을 만들어 본다....

반복번호 = 1
Epoch 1/1
70094/70094 [==============================] - 207s 3ms/step - loss: 9.0786

반복번호 = 2
Epoch 1/1
70094/70094 [==============================] - 203s 3ms/step - loss: 8.9883

반복번호 = 3
Epoch 1/1
70094/70094 [==============================] - 202s 3ms/step - loss: 8.8601

반복번호 = 4
Epoch 1/1
70094/70094 [==============================] - 204s 3ms/step - loss: 8.9178

반복번호 = 5
Epoch 1/1
70094/70094 [==============================] - 208s 3ms/step - loss: 8.9750

반복번호 = 6
Epoch 1/1
70094/70094 [==============================] - 202s 3ms/step - loss: 9.0034

반복번호 = 7
Epoch 1/1
70094/70094 [==============================] - 200s 3ms/step - loss: 9.0145

반복번호 = 8
Epoch 1/1
70094/70094 [==============================] - 201s 3ms/step - loss: 9.0236

반복번호 = 9
Epoch 1/1
70094/70094 [==============================] - 218s 3ms/step - loss: 9.0302


In [8]:
# 예측사건의 검증을 위한 적당한 threshold를 찾는다. 
next_probs = []

# 사건 일부 추출
all_s = [] 
for i, s in enumerate(sentences):
    #print(i,s)
    if i%15 == 0:
        all_s.append(s)


for ix, sentence in enumerate(all_s):
    print(ix, sentence)
    
    # 20개만 생성
    if ix > 20:
        break
    
    # 일련의 사건 -> 리스트로
    x = np.zeros((1, maxlen, len(chars)))
    list_sen = sentence.split(" ")
    enum_sen = enumerate(list_sen)

    for t, char in enum_sen:
        x[0, t, char_indices[char]] = 1.  # 한 일련의 사건
        # 다음 사건 예측
        preds = model.predict(x, verbose=0)[0]  # 다음 사건 인덱스
        next_pred_char = indices_char[np.argmax(preds)]  # 다음 사건 코드

        # 뒤에 사건이 비어있을 때만 예측
        if t < len(list_sen)-1:
            next_char = list_sen[t+1]
            next_prob = preds[char_indices[next_char]]
            next_probs.append(next_prob)
        else:
            next_char = "END"
            next_prob = 0

        print(t, char, preds, next_char,
              next_pred_char, next_prob,
              (next_prob - np.average(preds))>0)

0 No3368 En1339 Po1313 Te1354 En2411 No2563 Po3339 Bu2471 Ec3212 Se2421 En1314 En1442 Se3242 Tr3139 Tr2346 Se3233 En1434 No1366 No2127 No1433
0 No3368 [3.57029960e-04 7.49343890e-05 5.65882619e-05 ... 8.46810622e-07
 1.43240628e-04 1.08684646e-04] En1339 Te2164 7.993295e-05 False
1 En1339 [3.57066776e-04 7.50259205e-05 5.65282862e-05 ... 8.47067497e-07
 1.43471945e-04 1.08615015e-04] Po1313 Te2164 0.00023881419 True
2 Po1313 [3.5667996e-04 7.4924777e-05 5.6568490e-05 ... 8.4687326e-07 1.4326542e-04
 1.0862615e-04] Te1354 Te2164 0.00014069125 True
3 Te1354 [3.5633007e-04 7.4982607e-05 5.6445133e-05 ... 8.4646706e-07 1.4337873e-04
 1.0855397e-04] En2411 Te2164 0.00033546338 True
4 En2411 [3.55902303e-04 7.47609884e-05 5.68389441e-05 ... 8.47320223e-07
 1.42440826e-04 1.09145905e-04] No2563 Te2164 6.669831e-05 False
5 No2563 [3.5539307e-04 7.4900912e-05 5.6843208e-05 ... 8.4816804e-07 1.4232795e-04
 1.0940819e-04] Po3339 Te2164 0.00015937847 True
6 Po3339 [3.5571874e-04 7.4864955e-05 5.68

2 Ec1571 [3.5786239e-04 7.4934243e-05 5.6691868e-05 ... 8.4759603e-07 1.4325757e-04
 1.0876428e-04] Tr2441 Te2164 0.00031775018 True
3 Tr2441 [3.5735930e-04 7.5307980e-05 5.6469598e-05 ... 8.4862216e-07 1.4374433e-04
 1.0866607e-04] Ec2343 Te2164 0.00014960849 True
4 Ec2343 [3.5682775e-04 7.4694937e-05 5.6948822e-05 ... 8.4828810e-07 1.4262204e-04
 1.0916310e-04] Te3258 Te2164 5.8422993e-05 False
5 Te3258 [3.57836339e-04 7.47795930e-05 5.68341056e-05 ... 8.47160891e-07
 1.43187863e-04 1.08855966e-04] No3571 Te2164 6.1069186e-05 False
6 No3571 [3.5446492e-04 7.5113647e-05 5.6763034e-05 ... 8.4792543e-07 1.4256866e-04
 1.0954432e-04] Po3358 Te2164 8.795018e-05 False
7 Po3358 [3.5638874e-04 7.5014977e-05 5.6281868e-05 ... 8.4642073e-07 1.4443501e-04
 1.0795771e-04] No3427 Te2164 0.0001963711 True
8 No3427 [3.57688434e-04 7.46604783e-05 5.64498514e-05 ... 8.46268222e-07
 1.43788377e-04 1.08223154e-04] Te2442 Te2164 0.00013231281 True
9 Te2442 [3.57747514e-04 7.45065045e-05 5.72203317e-05 .

0 Se3444 [3.5660365e-04 7.4997959e-05 5.6558110e-05 ... 8.4705874e-07 1.4326455e-04
 1.0869679e-04] Po3333 Te2164 2.4116746e-06 False
1 Po3333 [3.5541665e-04 7.5050040e-05 5.6494715e-05 ... 8.4682125e-07 1.4301391e-04
 1.0872995e-04] Bu3556 Te2164 0.0001398279 True
2 Bu3556 [3.5622626e-04 7.5089403e-05 5.6418899e-05 ... 8.4684689e-07 1.4359801e-04
 1.0848384e-04] Tr2456 Te2164 0.0001977022 True
3 Tr2456 [3.56224482e-04 7.48176899e-05 5.64431539e-05 ... 8.45820864e-07
 1.43186495e-04 1.08443586e-04] Te3458 Te2164 0.00012547015 False
4 Te3458 [3.5694931e-04 7.4713680e-05 5.6400186e-05 ... 8.4470634e-07 1.4329460e-04
 1.0845242e-04] No2537 Te2164 4.2440453e-05 False
5 No2537 [3.5431696e-04 7.4931508e-05 5.6534947e-05 ... 8.4749388e-07 1.4252220e-04
 1.0888421e-04] Po3145 Te2164 1.7210741e-05 False
6 Po3145 [3.50820308e-04 7.56235895e-05 5.55687438e-05 ... 8.44724866e-07
 1.43309502e-04 1.08071334e-04] Tr2263 Te2164 0.00024711838 True
7 Tr2263 [3.4961803e-04 7.5089083e-05 5.6058027e-05 ...

4 No3314 [3.5579191e-04 7.5268261e-05 5.6304660e-05 ... 8.4805981e-07 1.4373567e-04
 1.0852780e-04] Ec1477 Te2164 6.7754656e-05 False
5 Ec1477 [3.5748412e-04 7.5217555e-05 5.6152010e-05 ... 8.4623338e-07 1.4450774e-04
 1.0808935e-04] Ec1378 Te2164 0.0002915486 True
6 Ec1378 [3.5635591e-04 7.4936564e-05 5.6559049e-05 ... 8.4751878e-07 1.4380050e-04
 1.0835476e-04] Po1257 Te2164 3.1184572e-05 False
7 Po1257 [3.5627195e-04 7.5377713e-05 5.6409128e-05 ... 8.5000772e-07 1.4441714e-04
 1.0853604e-04] Po3313 Te2164 0.00012570247 False
8 Po3313 [3.5036521e-04 7.6892538e-05 5.5935125e-05 ... 8.5660224e-07 1.4488141e-04
 1.0983886e-04] Ec2324 Te2164 0.00017796297 True
9 Ec2324 [3.5279730e-04 7.5387157e-05 5.6754001e-05 ... 8.5538068e-07 1.4472670e-04
 1.0859768e-04] Te3247 Te2164 0.00017705283 True
10 Te3247 [3.6013481e-04 7.5218733e-05 5.7578447e-05 ... 8.5906333e-07 1.4480769e-04
 1.0859216e-04] No1448 Te2164 0.00024016797 True
11 No1448 [3.4921977e-04 7.6504453e-05 5.8522332e-05 ... 8.5600959

6 No2579 [3.56697332e-04 7.46514052e-05 5.63549947e-05 ... 8.45447460e-07
 1.44231773e-04 1.08604516e-04] No1543 Te2164 0.00024439747 True
7 No1543 [3.5664783e-04 7.4979354e-05 5.6499557e-05 ... 8.4742868e-07 1.4486871e-04
 1.0891821e-04] Po2135 Te2164 0.0001673677 True
8 Po2135 [3.48190282e-04 7.41256517e-05 5.76914645e-05 ... 8.49601008e-07
 1.40340155e-04 1.11256886e-04] En3341 Te2164 6.0927825e-05 False
9 En3341 [3.4090897e-04 7.3984804e-05 5.7878824e-05 ... 8.5073111e-07 1.3796790e-04
 1.1225679e-04] Ec1177 Te2164 7.214879e-05 False
10 Ec1177 [3.4856552e-04 7.4343086e-05 5.8407008e-05 ... 8.5472857e-07 1.3903696e-04
 1.1280234e-04] Ec2538 Te2164 0.00016810032 True
11 Ec2538 [3.4679452e-04 7.4340023e-05 5.9108617e-05 ... 8.5382169e-07 1.3683058e-04
 1.1479887e-04] Tr2332 Te2164 0.00010078715 False
12 Tr2332 [3.4659862e-04 7.8088102e-05 5.6121407e-05 ... 8.5200907e-07 1.4681173e-04
 1.1068454e-04] Ec2273 Te2164 0.00016343239 True
13 Ec2273 [3.2614995e-04 7.9908961e-05 5.3815587e-05 

7 Po1271 [3.6152490e-04 7.4250129e-05 5.7143538e-05 ... 8.4735922e-07 1.4266490e-04
 1.0924236e-04] Tr2267 Te2164 0.00021559042 True
8 Tr2267 [3.54768621e-04 7.48130988e-05 5.59442124e-05 ... 8.40415794e-07
 1.44043006e-04 1.07545675e-04] Se3144 Bu2121 5.1163195e-05 False
9 Se3144 [3.4936992e-04 7.6325232e-05 5.5335313e-05 ... 8.4409191e-07 1.4613017e-04
 1.0736620e-04] Se2579 Bu2121 0.00031548858 True
10 Se2579 [3.5235798e-04 7.4968135e-05 5.6415516e-05 ... 8.4381662e-07 1.4258425e-04
 1.0943350e-04] Te3336 Te2164 8.4876e-05 False
11 Te3336 [3.4954303e-04 7.6176708e-05 5.5440079e-05 ... 8.4246528e-07 1.4667126e-04
 1.0789353e-04] No1555 Bu2121 2.8770386e-05 False
12 No1555 [3.4505039e-04 7.5958007e-05 5.6041372e-05 ... 8.3961737e-07 1.4300078e-04
 1.1061462e-04] Se1352 Te2164 0.00011474863 False
13 Se1352 [3.3738810e-04 7.7465498e-05 5.4802858e-05 ... 8.4958793e-07 1.4691288e-04
 1.1181403e-04] Bu1137 Te2164 0.00018529882 True
14 Bu1137 [3.1664563e-04 7.7405035e-05 6.1846767e-05 ... 8

1 En1251 [3.5665400e-04 7.4775562e-05 5.6697980e-05 ... 8.4670006e-07 1.4288251e-04
 1.0883827e-04] Te3536 Te2164 0.00011537392 False
2 Te3536 [3.5635271e-04 7.4731783e-05 5.6716694e-05 ... 8.4675656e-07 1.4275582e-04
 1.0885300e-04] Se1235 Te2164 8.523364e-05 False
3 Se1235 [3.56041361e-04 7.48458624e-05 5.65773844e-05 ... 8.46567161e-07
 1.43076031e-04 1.08698994e-04] Se1123 Te2164 9.584856e-06 False
4 Se1123 [3.5614124e-04 7.4881704e-05 5.6613881e-05 ... 8.4708086e-07 1.4291295e-04
 1.0867807e-04] Te1536 Te2164 0.00013677667 True
5 Te1536 [3.5310700e-04 7.5426855e-05 5.5845241e-05 ... 8.4605239e-07 1.4393966e-04
 1.0789653e-04] Bu1539 Te2164 5.3787753e-05 False
6 Bu1539 [3.5124493e-04 7.5244214e-05 5.6333058e-05 ... 8.4701611e-07 1.4227541e-04
 1.0882470e-04] En1348 Te2164 6.593996e-05 False
7 En1348 [3.4977988e-04 7.5233394e-05 5.6457477e-05 ... 8.4634058e-07 1.4163174e-04
 1.0922785e-04] Se2314 Te2164 9.872526e-05 False
8 Se2314 [3.4624359e-04 7.5293174e-05 5.6487363e-05 ... 8.478

In [9]:
# 일어난 사건을 검증하여 본다. 
sentence = "Tr3317 Po2149 Te3473 En2146"
x = np.zeros((1, maxlen, len(chars)))
list_sen = sentence.split(" ")
enum_sen = enumerate(list_sen)
err_word_all = []
real_word_all = []

In [10]:
# 일반적으로 일어나는 사건의 확률의 정도를 감안하기 위하여 평균을 구하여 본다. 
for t, char in enum_sen:
    x[0, t, char_indices[char]] = 1.
    preds = model.predict(x, verbose=0)[0]
    next_pred_char = indices_char[np.argmax(preds)]

    if t < len(list_sen)-1:
        next_char = list_sen[t+1]
        next_prob = preds[char_indices[next_char]]
    else:
        next_char = "END"
        next_prob = 0

    diff = (next_prob - np.average(next_probs))
    
    if diff < 0:
        err_word = char + " " + next_char 
        err_word_all.append(err_word)
    elif diff > 0:
        real_word = char + " " + next_char
        real_word_all.append(real_word)
    
    print(t , char , next_char , next_prob , diff , diff > 0)

print(real_word_all, err_word_all[:-1])


0 Tr3317 Po2149 0.00012405444 -1.0858217e-05 False
1 Po2149 Te3473 0.00023035394 9.544128e-05 True
2 Te3473 En2146 4.622332e-05 -8.868934e-05 False
3 En2146 END 0 -0.00013491265417542309 False
['Po2149 Te3473'] ['Tr3317 Po2149', 'Te3473 En2146']
